<a href="https://colab.research.google.com/github/BimalGhimire38/Extra_Python_Stuffs/blob/main/Extracting_English_lessons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing for IELTS
2082/08/27  07:38 AM

In [1]:
!pip install requests beautifulsoup4 fpdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 19.1 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import time

def get_topics(main_url):
    """
    Scrape the main page to get list of topics and their URLs.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(main_url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')
    ol = soup.find('ol')
    if not ol:
        raise ValueError("Could not find the ordered list of topics.")

    topics = []
    for li in ol.find_all('li'):
        a = li.find('a')
        if a:
            title = a.get_text().strip()
            url = a['href']
            if not url.startswith('http'):
                url = 'https://basicenglishspeaking.com' + url
            topics.append((title, url))

    print(f"Found {len(topics)} topics.")
    return topics

def get_lesson_content(lesson_url):
    """
    Scrape a single lesson page to extract question-answer pairs.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(lesson_url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')
    content_div = soup.find('div', class_='entry-content')
    if not content_div:
        raise ValueError(f"Could not find content div for {lesson_url}")

    paragraphs = content_div.find_all('p')
    questions_answers = []

    # Assume pairs: odd index question, even answer (0-based)
    for i in range(0, len(paragraphs), 2):
        if i + 1 < len(paragraphs):
            question = paragraphs[i].get_text().strip()
            # Filter out empty or non-question paragraphs
            if question.endswith('?') and len(question) > 10:
                answer = paragraphs[i + 1].get_text().strip()
                if answer and not answer.endswith('?'):
                    questions_answers.append((question, answer))

    return questions_answers

def create_pdf(topics, output_file='all_english_lessons.pdf'):
    """
    Generate a single PDF with all lessons.
    """
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)

    cover_title = "75 Daily English Conversation Lessons"
    pdf.cell(0, 10, txt=cover_title, ln=True, align='C')
    pdf.ln(10)

    pdf.set_font('Arial', '', 12)

    lesson_count = 0
    for title, url in topics:
        lesson_count += 1
        print(f"Processing lesson {lesson_count}: {title}")

        # Add title with bold
        pdf.set_font('Arial', 'B', 14)
        pdf.cell(0, 10, txt=title, ln=True, align='C')
        pdf.ln(5)

        # Reset to normal font
        pdf.set_font('Arial', '', 11)

        try:
            qa_pairs = get_lesson_content(url)
            for q, a in qa_pairs:
                # Question bold
                pdf.set_font('Arial', 'B')
                pdf.multi_cell(0, 6, f"Q: {q}")

                # Answer normal
                pdf.set_font('Arial', '')
                pdf.multi_cell(0, 6, f"A: {a}")
                pdf.ln(3)

            # Page break after every 5 lessons or if page is full
            if lesson_count % 5 == 0:
                pdf.add_page()

        except Exception as e:
            print(f"Error processing {title}: {e}")
            pdf.set_font('Arial', '')
            pdf.cell(0, 10, txt=f"Error loading content for {title}", ln=True)

        time.sleep(1)  # Be polite to the server

    pdf.output(output_file)
    print(f"PDF generated: {output_file}")

if __name__ == "__main__":
    main_url = "https://basicenglishspeaking.com/daily-english-conversation-topics/"
    topics = get_topics(main_url)
    create_pdf(topics)

ValueError: Could not find the ordered list of topics.

In [3]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import time
import re

def get_topics(main_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(main_url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all <a> tags that look like lesson links (href ends with /word-or-phrase/)
    lesson_links = soup.find_all('a', href=re.compile(r'/[a-zA-Z-]+$'))

    topics = []
    seen = set()  # Avoid duplicates
    for a in lesson_links:
        url = a['href']
        if not url.startswith('http'):
            url = 'https://basicenglishspeaking.com' + url if url.startswith('/') else 'https://basicenglishspeaking.com/' + url
        title = a.get_text().strip()
        if title and url not in seen and len(title) > 1:
            topics.append((title, url))
            seen.add(url)

    # Sort by approximate number if possible (fallback to as-is)
    def sort_key(item):
        text = item[0] + ' ' + item[1]
        numbers = re.search(r'(\d+)', text)
        return int(numbers.group(1)) if numbers else 999

    topics.sort(key=sort_key)

    print(f"Found {len(topics)} topics.")
    if len(topics) < 70:
        print("Warning: Fewer topics found than expected. Site may have changed.")
    return topics

def get_lesson_content(lesson_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(lesson_url, headers=headers, timeout=10)
        response.raise_for_status()
    except Exception as e:
        raise ValueError(f"Failed to load {lesson_url}: {e}")

    soup = BeautifulSoup(response.content, 'html.parser')
    content_div = soup.find('div', class_='entry-content')
    if not content_div:
        raise ValueError(f"Could not find content div for {lesson_url}")

    paragraphs = content_div.find_all('p')
    questions_answers = []

    i = 0
    while i < len(paragraphs) - 1:
        q_para = paragraphs[i].get_text(separator=' ').strip()
        a_para = paragraphs[i+1].get_text(separator=' ').strip()

        if q_para.endswith('?') and len(q_para) > 10 and a_para and not a_para.endswith('?'):
            questions_answers.append((q_para, a_para))
            i += 2
        else:
            i += 1  # Skip if not a clear Q-A pair

    return questions_answers

def create_pdf(topics, output_file='all_english_lessons.pdf'):
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, txt="75 Daily English Conversation Lessons", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font('Arial', '', 12)

    lesson_count = 0
    for title, url in topics:
        lesson_count += 1
        print(f"Processing lesson {lesson_count}: {title} ({url})")

        pdf.set_font('Arial', 'B', 14)
        pdf.cell(0, 10, txt=f"{lesson_count}. {title}", ln=True, align='C')
        pdf.ln(5)

        pdf.set_font('Arial', '', 11)

        try:
            qa_pairs = get_lesson_content(url)
            if not qa_pairs:
                pdf.multi_cell(0, 6, "No Q&A content found.")
            for q, a in qa_pairs:
                pdf.set_font('Arial', 'B')
                pdf.multi_cell(0, 6, f"Q: {q}")

                pdf.set_font('Arial', '')
                pdf.multi_cell(0, 6, f"A: {a}")
                pdf.ln(3)

            if lesson_count % 4 == 0:  # Add page break every few lessons to avoid overflow
                pdf.add_page()

        except Exception as e:
            print(f"Error processing {title}: {e}")
            pdf.set_font('Arial', 'I')
            pdf.multi_cell(0, 6, f"Error loading content: {str(e)}")

        time.sleep(1.5)  # Increased delay to be extra polite to the server

    pdf.output(output_file)
    print(f"PDF generated successfully: {output_file}")

if __name__ == "__main__":
    main_url = "https://basicenglishspeaking.com/daily-english-conversation-topics/"
    topics = get_topics(main_url)
    create_pdf(topics)

Found 0 topics.
PDF generated successfully: all_english_lessons.pdf


/tmp/ipython-input-3847116682.py:77: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
/tmp/ipython-input-3847116682.py:78: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(0, 10, txt="75 Daily English Conversation Lessons", ln=True, align='C')
/tmp/ipython-input-3847116682.py:78: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, txt="75 Daily English Conversation Lessons", ln=True, align='C')
/tmp/ipython-input-3847116682.py:80: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 12)


In [4]:
!pip install requests beautifulsoup4 fpdf2

import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import time

# Hardcoded list of all 75 topics with titles and URLs (accurate as of now)
topics = [
    ("Family", "https://basicenglishspeaking.com/family/"),
    ("Restaurant", "https://basicenglishspeaking.com/restaurant/"),
    ("Books", "https://basicenglishspeaking.com/books/"),
    ("Travel", "https://basicenglishspeaking.com/travel/"),
    ("Website", "https://basicenglishspeaking.com/website/"),
    ("Accident", "https://basicenglishspeaking.com/accident/"),
    ("Childhood memory", "https://basicenglishspeaking.com/childhood-memory/"),
    ("Favorite rooms", "https://basicenglishspeaking.com/favorite-rooms/"),
    ("Presents", "https://basicenglishspeaking.com/presents/"),
    ("Historical place", "https://basicenglishspeaking.com/historical-place/"),
    ("Newspaper/ Magazine", "https://basicenglishspeaking.com/newspaper-magazine/"),
    ("A memorable event", "https://basicenglishspeaking.com/an-event/"),
    ("A favorite subject", "https://basicenglishspeaking.com/a-subject/"),
    ("A museum", "https://basicenglishspeaking.com/a-museum/"),
    ("A favorite movie", "https://basicenglishspeaking.com/a-movie/"),
    ("A foreign country", "https://basicenglishspeaking.com/a-foreign-country/"),
    ("Parties", "https://basicenglishspeaking.com/parties/"),
    ("A teacher", "https://basicenglishspeaking.com/a-teacher/"),
    ("A friend", "https://basicenglishspeaking.com/a-friend/"),
    ("A hotel", "https://basicenglishspeaking.com/a-hotel/"),
    ("A letter", "https://basicenglishspeaking.com/a-letter/"),
    ("Hobbies", "https://basicenglishspeaking.com/hobbies/"),
    ("Music", "https://basicenglishspeaking.com/music/"),
    ("Shopping", "https://basicenglishspeaking.com/shopping/"),
    ("Holiday", "https://basicenglishspeaking.com/holiday/"),
    ("Animals", "https://basicenglishspeaking.com/animals/"),
    ("A practical skill", "https://basicenglishspeaking.com/a-practical-skill/"),
    ("Sport", "https://basicenglishspeaking.com/sports/"),
    ("A School", "https://basicenglishspeaking.com/a-school/"),
    ("Festival", "https://basicenglishspeaking.com/festival/"),
    ("Food", "https://basicenglishspeaking.com/food/"),
    ("Household appliance", "https://basicenglishspeaking.com/household-appliance/"),
    ("A music band", "https://basicenglishspeaking.com/a-music-band/"),
    ("Weather", "https://basicenglishspeaking.com/weather/"),
    ("Neighbor", "https://basicenglishspeaking.com/neighbor/"),
    ("Natural scenery", "https://basicenglishspeaking.com/natural-scenery/"),
    ("Outdoor activities", "https://basicenglishspeaking.com/outdoor-activities/"),
    ("Law", "https://basicenglishspeaking.com/law/"),
    ("Pollution", "https://basicenglishspeaking.com/pollution/"),
    ("Traffic jam", "https://basicenglishspeaking.com/traffic-jam/"),
    ("TV program", "https://basicenglishspeaking.com/tv-program/"),
    ("Architect/ Building", "https://basicenglishspeaking.com/architecture-building/"),
    ("Electronic Media", "https://basicenglishspeaking.com/electronic-media/"),
    ("Job/ Career", "https://basicenglishspeaking.com/job-career/"),
    ("Competition/ contest", "https://basicenglishspeaking.com/competition-contest/"),
    ("A garden", "https://basicenglishspeaking.com/a-garden/"),
    ("Hometown", "https://basicenglishspeaking.com/hometown/"),
    ("Clothing", "https://basicenglishspeaking.com/clothing/"),
    ("Advertisement", "https://basicenglishspeaking.com/an-advertisement/"),
    ("A project", "https://basicenglishspeaking.com/a-project/"),
    ("A wedding", "https://basicenglishspeaking.com/a-wedding/"),
    ("A Coffee shop", "https://basicenglishspeaking.com/a-coffee-shop/"),
    ("Culture", "https://basicenglishspeaking.com/culture/"),
    ("Transport", "https://basicenglishspeaking.com/transport/"),
    ("Politician", "https://basicenglishspeaking.com/politician/"),
    ("Communication", "https://basicenglishspeaking.com/communication/"),
    ("Business", "https://basicenglishspeaking.com/business/"),
    ("Computer", "https://basicenglishspeaking.com/computer/"),
    ("Exercise", "https://basicenglishspeaking.com/exercise/"),
    ("Goal/ ambition", "https://basicenglishspeaking.com/goal-ambition/"),
    ("Art", "https://basicenglishspeaking.com/art/"),
    ("Fashion", "https://basicenglishspeaking.com/fashion/"),
    ("Jewelry", "https://basicenglishspeaking.com/jewelry/"),
    ("Cosmetic", "https://basicenglishspeaking.com/cosmetic/"),
    ("Indoor Game", "https://basicenglishspeaking.com/indoor-game/"),
    ("Phone conversation", "https://basicenglishspeaking.com/phone-conversation/"),
    ("Learning A Second language", "https://basicenglishspeaking.com/learning-second-language/"),
    ("A Creative Person", "https://basicenglishspeaking.com/a-creative-person/"),
    ("A celebrity", "https://basicenglishspeaking.com/a-celebrity/"),
    ("A Health Problem", "https://basicenglishspeaking.com/a-health-problem/"),
    ("Technological advancements", "https://basicenglishspeaking.com/technological-advancements/"),
    ("A Landmark", "https://basicenglishspeaking.com/a-landmark/"),
    ("Handcraft Items", "https://basicenglishspeaking.com/handcraft-items/"),
    ("Plastic Surgery", "https://basicenglishspeaking.com/plastic-surgery/"),
    ("Success", "https://basicenglishspeaking.com/success/"),
]

def get_lesson_content(lesson_url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    try:
        response = requests.get(lesson_url, headers=headers, timeout=15)
        response.raise_for_status()
    except Exception as e:
        raise ValueError(f"Failed to load {lesson_url}: {e}")

    soup = BeautifulSoup(response.content, 'html.parser')
    content_div = soup.find('div', class_='entry-content')
    if not content_div:
        raise ValueError("No entry-content div found")

    paragraphs = [p.get_text(separator=' ').strip() for p in content_div.find_all('p')]

    qa_pairs = []
    i = 0
    while i < len(paragraphs) - 1:
        q = paragraphs[i]
        a = paragraphs[i+1]
        if q.endswith('?') and len(q) > 10 and a and not a.endswith('?'):
            qa_pairs.append((q, a))
            i += 2
        else:
            i += 1
    return qa_pairs

def create_pdf(topics, output_file='all_english_lessons.pdf'):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Helvetica", 'B', 16)  # Use Helvetica to avoid Arial warnings
    pdf.cell(0, 10, "75 Daily English Conversation Lessons", ln=1, align='C')
    pdf.ln(10)

    for idx, (title, url) in enumerate(topics, 1):
        print(f"Processing {idx}/75: {title}")

        pdf.set_font("Helvetica", 'B', 14)
        pdf.cell(0, 10, f"{idx}. {title}", ln=1, align='C')
        pdf.ln(5)

        pdf.set_font("Helvetica", '', 11)

        try:
            qa_pairs = get_lesson_content(url)
            if not qa_pairs:
                pdf.multi_cell(0, 6, "No Q&A content found.")
            for q, a in qa_pairs:
                pdf.set_font("Helvetica", 'B')
                pdf.multi_cell(0, 6, f"Q: {q}")
                pdf.set_font("Helvetica", '')
                pdf.multi_cell(0, 6, f"A: {a}")
                pdf.ln(3)
            if idx % 4 == 0:
                pdf.add_page()
        except Exception as e:
            print(f"Error on {title}: {e}")
            pdf.set_font("Helvetica", 'I')
            pdf.multi_cell(0, 6, f"Error: Could not load content ({str(e)})")

        time.sleep(1.5)  # Polite delay

    pdf.output(output_file)
    print(f"PDF generated: {output_file}")

create_pdf(topics)

from google.colab import files
files.download('all_english_lessons.pdf')

/tmp/ipython-input-2927125882.py:118: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "75 Daily English Conversation Lessons", ln=1, align='C')
/tmp/ipython-input-2927125882.py:125: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f"{idx}. {title}", ln=1, align='C')


Processing 1/75: Family
Error on Family: No entry-content div found
Processing 2/75: Restaurant
Error on Restaurant: No entry-content div found
Processing 3/75: Books
Error on Books: No entry-content div found
Processing 4/75: Travel
Error on Travel: No entry-content div found
Processing 5/75: Website
Error on Website: No entry-content div found
Processing 6/75: Accident
Error on Accident: No entry-content div found
Processing 7/75: Childhood memory
Error on Childhood memory: No entry-content div found
Processing 8/75: Favorite rooms
Error on Favorite rooms: No entry-content div found
Processing 9/75: Presents
Error on Presents: No entry-content div found
Processing 10/75: Historical place
Error on Historical place: No entry-content div found
Processing 11/75: Newspaper/ Magazine
Error on Newspaper/ Magazine: No entry-content div found
Processing 12/75: A memorable event
Error on A memorable event: No entry-content div found
Processing 13/75: A favorite subject
Error on A favorite subj

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
!pip install requests beautifulsoup4 fpdf2

import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import time

# Hardcoded list of all 75 topics (titles and URLs) - confirmed current
topics = [
    ("Family", "https://basicenglishspeaking.com/family/"),
    ("Restaurant", "https://basicenglishspeaking.com/restaurant/"),
    ("Books", "https://basicenglishspeaking.com/books/"),
    ("Travel", "https://basicenglishspeaking.com/travel/"),
    ("Website", "https://basicenglishspeaking.com/website/"),
    ("Accident", "https://basicenglishspeaking.com/accident/"),
    ("Childhood memory", "https://basicenglishspeaking.com/childhood-memory/"),
    ("Favorite rooms", "https://basicenglishspeaking.com/favorite-rooms/"),
    ("Presents", "https://basicenglishspeaking.com/presents/"),
    ("Historical place", "https://basicenglishspeaking.com/historical-place/"),
    ("Newspaper/ Magazine", "https://basicenglishspeaking.com/newspaper-magazine/"),
    ("A memorable event", "https://basicenglishspeaking.com/an-event/"),
    ("A favorite subject", "https://basicenglishspeaking.com/a-subject/"),
    ("A museum", "https://basicenglishspeaking.com/a-museum/"),
    ("A favorite movie", "https://basicenglishspeaking.com/a-movie/"),
    ("A foreign country", "https://basicenglishspeaking.com/a-foreign-country/"),
    ("Parties", "https://basicenglishspeaking.com/parties/"),
    ("A teacher", "https://basicenglishspeaking.com/a-teacher/"),
    ("A friend", "https://basicenglishspeaking.com/a-friend/"),
    ("A hotel", "https://basicenglishspeaking.com/a-hotel/"),
    ("A letter", "https://basicenglishspeaking.com/a-letter/"),
    ("Hobbies", "https://basicenglishspeaking.com/hobbies/"),
    ("Music", "https://basicenglishspeaking.com/music/"),
    ("Shopping", "https://basicenglishspeaking.com/shopping/"),
    ("Holiday", "https://basicenglishspeaking.com/holiday/"),
    ("Animals", "https://basicenglishspeaking.com/animals/"),
    ("A practical skill", "https://basicenglishspeaking.com/a-practical-skill/"),
    ("Sport", "https://basicenglishspeaking.com/sports/"),
    ("A School", "https://basicenglishspeaking.com/a-school/"),
    ("Festival", "https://basicenglishspeaking.com/festival/"),
    ("Food", "https://basicenglishspeaking.com/food/"),
    ("Household appliance", "https://basicenglishspeaking.com/household-appliance/"),
    ("A music band", "https://basicenglishspeaking.com/a-music-band/"),
    ("Weather", "https://basicenglishspeaking.com/weather/"),
    ("Neighbor", "https://basicenglishspeaking.com/neighbor/"),
    ("Natural scenery", "https://basicenglishspeaking.com/natural-scenery/"),
    ("Outdoor activities", "https://basicenglishspeaking.com/outdoor-activities/"),
    ("Law", "https://basicenglishspeaking.com/law/"),
    ("Pollution", "https://basicenglishspeaking.com/pollution/"),
    ("Traffic jam", "https://basicenglishspeaking.com/traffic-jam/"),
    ("TV program", "https://basicenglishspeaking.com/tv-program/"),
    ("Architect/ Building", "https://basicenglishspeaking.com/architecture-building/"),
    ("Electronic Media", "https://basicenglishspeaking.com/electronic-media/"),
    ("Job/ Career", "https://basicenglishspeaking.com/job-career/"),
    ("Competition/ contest", "https://basicenglishspeaking.com/competition-contest/"),
    ("A garden", "https://basicenglishspeaking.com/a-garden/"),
    ("Hometown", "https://basicenglishspeaking.com/hometown/"),
    ("Clothing", "https://basicenglishspeaking.com/clothing/"),
    ("Advertisement", "https://basicenglishspeaking.com/an-advertisement/"),
    ("A project", "https://basicenglishspeaking.com/a-project/"),
    ("A wedding", "https://basicenglishspeaking.com/a-wedding/"),
    ("A Coffee shop", "https://basicenglishspeaking.com/a-coffee-shop/"),
    ("Culture", "https://basicenglishspeaking.com/culture/"),
    ("Transport", "https://basicenglishspeaking.com/transport/"),
    ("Politician", "https://basicenglishspeaking.com/politician/"),
    ("Communication", "https://basicenglishspeaking.com/communication/"),
    ("Business", "https://basicenglishspeaking.com/business/"),
    ("Computer", "https://basicenglishspeaking.com/computer/"),
    ("Exercise", "https://basicenglishspeaking.com/exercise/"),
    ("Goal/ ambition", "https://basicenglishspeaking.com/goal-ambition/"),
    ("Art", "https://basicenglishspeaking.com/art/"),
    ("Fashion", "https://basicenglishspeaking.com/fashion/"),
    ("Jewelry", "https://basicenglishspeaking.com/jewelry/"),
    ("Cosmetic", "https://basicenglishspeaking.com/cosmetic/"),
    ("Indoor Game", "https://basicenglishspeaking.com/indoor-game/"),
    ("Phone conversation", "https://basicenglishspeaking.com/phone-conversation/"),
    ("Learning A Second language", "https://basicenglishspeaking.com/learning-second-language/"),
    ("A Creative Person", "https://basicenglishspeaking.com/a-creative-person/"),
    ("A celebrity", "https://basicenglishspeaking.com/a-celebrity/"),
    ("A Health Problem", "https://basicenglishspeaking.com/a-health-problem/"),
    ("Technological advancements", "https://basicenglishspeaking.com/technological-advancements/"),
    ("A Landmark", "https://basicenglishspeaking.com/a-landmark/"),
    ("Handcraft Items", "https://basicenglishspeaking.com/handcraft-items/"),
    ("Plastic Surgery", "https://basicenglishspeaking.com/plastic-surgery/"),
    ("Success", "https://basicenglishspeaking.com/success/"),
]

def get_lesson_content(lesson_url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    try:
        response = requests.get(lesson_url, headers=headers, timeout=15)
        response.raise_for_status()
    except Exception as e:
        raise ValueError(f"Failed to load page: {e}")

    soup = BeautifulSoup(response.content, 'html.parser')

    # Try to find main content article
    article = soup.find('article') or soup.find('div', class_='post-content') or soup.find('main')
    if article:
        paragraphs = [p.get_text(separator=' ').strip() for p in article.find_all('p') if p.get_text(strip=True)]
    else:
        # Fallback: all <p> in body
        paragraphs = [p.get_text(separator=' ').strip() for p in soup.find_all('p') if p.get_text(strip=True)]

    # Filter out non-QA paragraphs (e.g., intro, audio links)
    qa_paras = []
    for para in paragraphs:
        if len(para) > 20:  # Reasonable length
            qa_paras.append(para)

    qa_pairs = []
    i = 0
    while i < len(qa_paras) - 1:
        q = qa_paras[i]
        a = qa_paras[i+1]
        # Check if question ends with ? and answer doesn't
        if q.endswith('?') and len(q) > 10 and a and not a.endswith('?'):
            qa_pairs.append((q.strip(), a.strip()))
            i += 2
        else:
            i += 1  # Skip potential non-pair

    return qa_pairs

def create_pdf(topics, output_file='all_english_lessons.pdf'):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Helvetica", 'B', 16)
    pdf.cell(0, 10, "75 Daily English Conversation Lessons", ln=1, align='C')
    pdf.ln(10)

    for idx, (title, url) in enumerate(topics, 1):
        print(f"Processing {idx}/75: {title}")

        pdf.set_font("Helvetica", 'B', 14)
        pdf.cell(0, 10, f"{idx}. {title}", ln=1, align='C')
        pdf.ln(5)

        pdf.set_font("Helvetica", '', 11)

        try:
            qa_pairs = get_lesson_content(url)
            if not qa_pairs:
                pdf.multi_cell(0, 6, "No Q&A content extracted (possible site protection).")
            for q, a in qa_pairs:
                pdf.set_font("Helvetica", 'B')
                pdf.multi_cell(0, 6, f"Q: {q}")
                pdf.set_font("Helvetica", '')
                pdf.multi_cell(0, 6, f"A: {a}")
                pdf.ln(3)
            if idx % 4 == 0:
                pdf.add_page()
        except Exception as e:
            print(f"Error on {title}: {e}")
            pdf.set_font("Helvetica", 'I')
            pdf.multi_cell(0, 6, f"Error loading content: {str(e)}")

        time.sleep(2)  # Increased delay to reduce risk of blocking

    pdf.output(output_file)
    print(f"PDF generated: {output_file}")

create_pdf(topics)

from google.colab import files
files.download('all_english_lessons.pdf')

/tmp/ipython-input-1319699784.py:129: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "75 Daily English Conversation Lessons", ln=1, align='C')
/tmp/ipython-input-1319699784.py:136: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f"{idx}. {title}", ln=1, align='C')


Processing 1/75: Family
Processing 2/75: Restaurant
Processing 3/75: Books
Processing 4/75: Travel
Processing 5/75: Website
Error on Website: Failed to load page: 429 Client Error: Too Many Requests for url: https://basicenglishspeaking.com/website/
Processing 6/75: Accident
Processing 7/75: Childhood memory
Processing 8/75: Favorite rooms
Processing 9/75: Presents
Processing 10/75: Historical place
Processing 11/75: Newspaper/ Magazine
Processing 12/75: A memorable event
Processing 13/75: A favorite subject
Processing 14/75: A museum
Processing 15/75: A favorite movie
Processing 16/75: A foreign country
Processing 17/75: Parties
Processing 18/75: A teacher
Processing 19/75: A friend
Processing 20/75: A hotel
Processing 21/75: A letter
Processing 22/75: Hobbies
Processing 23/75: Music
Processing 24/75: Shopping
Processing 25/75: Holiday
Processing 26/75: Animals
Processing 27/75: A practical skill
Processing 28/75: Sport
Processing 29/75: A School
Processing 30/75: Festival
Processing 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Now Using Selenium

In [6]:
# Install dependencies for Selenium in Colab
!apt-get update -qq
!apt install -qq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium beautifulsoup4 fpdf2

import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from fpdf import FPDF

# Hardcoded list of 75 topics (same as before)
topics = [
    ("Family", "https://basicenglishspeaking.com/family/"),
    ("Restaurant", "https://basicenglishspeaking.com/restaurant/"),
    ("Books", "https://basicenglishspeaking.com/books/"),
    ("Travel", "https://basicenglishspeaking.com/travel/"),
    ("Website", "https://basicenglishspeaking.com/website/"),
    ("Accident", "https://basicenglishspeaking.com/accident/"),
    ("Childhood memory", "https://basicenglishspeaking.com/childhood-memory/"),
    ("Favorite rooms", "https://basicenglishspeaking.com/favorite-rooms/"),
    ("Presents", "https://basicenglishspeaking.com/presents/"),
    ("Historical place", "https://basicenglishspeaking.com/historical-place/"),
    ("Newspaper/ Magazine", "https://basicenglishspeaking.com/newspaper-magazine/"),
    ("A memorable event", "https://basicenglishspeaking.com/an-event/"),
    ("A favorite subject", "https://basicenglishspeaking.com/a-subject/"),
    ("A museum", "https://basicenglishspeaking.com/a-museum/"),
    ("A favorite movie", "https://basicenglishspeaking.com/a-movie/"),
    ("A foreign country", "https://basicenglishspeaking.com/a-foreign-country/"),
    ("Parties", "https://basicenglishspeaking.com/parties/"),
    ("A teacher", "https://basicenglishspeaking.com/a-teacher/"),
    ("A friend", "https://basicenglishspeaking.com/a-friend/"),
    ("A hotel", "https://basicenglishspeaking.com/a-hotel/"),
    ("A letter", "https://basicenglishspeaking.com/a-letter/"),
    ("Hobbies", "https://basicenglishspeaking.com/hobbies/"),
    ("Music", "https://basicenglishspeaking.com/music/"),
    ("Shopping", "https://basicenglishspeaking.com/shopping/"),
    ("Holiday", "https://basicenglishspeaking.com/holiday/"),
    ("Animals", "https://basicenglishspeaking.com/animals/"),
    ("A practical skill", "https://basicenglishspeaking.com/a-practical-skill/"),
    ("Sport", "https://basicenglishspeaking.com/sports/"),
    ("A School", "https://basicenglishspeaking.com/a-school/"),
    ("Festival", "https://basicenglishspeaking.com/festival/"),
    ("Food", "https://basicenglishspeaking.com/food/"),
    ("Household appliance", "https://basicenglishspeaking.com/household-appliance/"),
    ("A music band", "https://basicenglishspeaking.com/a-music-band/"),
    ("Weather", "https://basicenglishspeaking.com/weather/"),
    ("Neighbor", "https://basicenglishspeaking.com/neighbor/"),
    ("Natural scenery", "https://basicenglishspeaking.com/natural-scenery/"),
    ("Outdoor activities", "https://basicenglishspeaking.com/outdoor-activities/"),
    ("Law", "https://basicenglishspeaking.com/law/"),
    ("Pollution", "https://basicenglishspeaking.com/pollution/"),
    ("Traffic jam", "https://basicenglishspeaking.com/traffic-jam/"),
    ("TV program", "https://basicenglishspeaking.com/tv-program/"),
    ("Architect/ Building", "https://basicenglishspeaking.com/architecture-building/"),
    ("Electronic Media", "https://basicenglishspeaking.com/electronic-media/"),
    ("Job/ Career", "https://basicenglishspeaking.com/job-career/"),
    ("Competition/ contest", "https://basicenglishspeaking.com/competition-contest/"),
    ("A garden", "https://basicenglishspeaking.com/a-garden/"),
    ("Hometown", "https://basicenglishspeaking.com/hometown/"),
    ("Clothing", "https://basicenglishspeaking.com/clothing/"),
    ("Advertisement", "https://basicenglishspeaking.com/an-advertisement/"),
    ("A project", "https://basicenglishspeaking.com/a-project/"),
    ("A wedding", "https://basicenglishspeaking.com/a-wedding/"),
    ("A Coffee shop", "https://basicenglishspeaking.com/a-coffee-shop/"),
    ("Culture", "https://basicenglishspeaking.com/culture/"),
    ("Transport", "https://basicenglishspeaking.com/transport/"),
    ("Politician", "https://basicenglishspeaking.com/politician/"),
    ("Communication", "https://basicenglishspeaking.com/communication/"),
    ("Business", "https://basicenglishspeaking.com/business/"),
    ("Computer", "https://basicenglishspeaking.com/computer/"),
    ("Exercise", "https://basicenglishspeaking.com/exercise/"),
    ("Goal/ ambition", "https://basicenglishspeaking.com/goal-ambition/"),
    ("Art", "https://basicenglishspeaking.com/art/"),
    ("Fashion", "https://basicenglishspeaking.com/fashion/"),
    ("Jewelry", "https://basicenglishspeaking.com/jewelry/"),
    ("Cosmetic", "https://basicenglishspeaking.com/cosmetic/"),
    ("Indoor Game", "https://basicenglishspeaking.com/indoor-game/"),
    ("Phone conversation", "https://basicenglishspeaking.com/phone-conversation/"),
    ("Learning A Second language", "https://basicenglishspeaking.com/learning-second-language/"),
    ("A Creative Person", "https://basicenglishspeaking.com/a-creative-person/"),
    ("A celebrity", "https://basicenglishspeaking.com/a-celebrity/"),
    ("A Health Problem", "https://basicenglishspeaking.com/a-health-problem/"),
    ("Technological advancements", "https://basicenglishspeaking.com/technological-advancements/"),
    ("A Landmark", "https://basicenglishspeaking.com/a-landmark/"),
    ("Handcraft Items", "https://basicenglishspeaking.com/handcraft-items/"),
    ("Plastic Surgery", "https://basicenglishspeaking.com/plastic-surgery/"),
    ("Success", "https://basicenglishspeaking.com/success/"),
]

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def get_lesson_content(driver, lesson_url):
    driver.get(lesson_url)
    time.sleep(random.uniform(3, 5))  # Random wait to mimic human

    # Wait for content to load
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "entry-content"))
        )
    except:
        pass  # Continue if not found

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find content area
    content = soup.find('div', class_='entry-content') or soup.find('article')
    if not content:
        return []

    paragraphs = [p.get_text(separator=' ').strip() for p in content.find_all('p') if p.get_text(strip=True) and len(p.get_text(strip=True)) > 10]

    qa_pairs = []
    i = 0
    while i < len(paragraphs) - 1:
        q = paragraphs[i].strip()
        a = paragraphs[i + 1].strip()
        if q.endswith('?') and len(q) > 5 and a and not a.endswith('?'):
            qa_pairs.append((q, a))
            i += 2
        else:
            i += 1

    return qa_pairs

def create_pdf(topics, output_file='all_english_lessons.pdf'):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Helvetica", 'B', 16)
    pdf.cell(0, 10, text="75 Daily English Conversation Lessons", new_x=FPDF.LMARGIN, new_y=FPDF.NEXT, align='C')
    pdf.ln(10)

    driver = setup_driver()
    try:
        for idx, (title, url) in enumerate(topics, 1):
            print(f"Processing {idx}/75: {title}")

            pdf.set_font("Helvetica", 'B', 14)
            pdf.cell(0, 10, text=f"{idx}. {title}", new_x=FPDF.LMARGIN, new_y=FPDF.NEXT, align='C')
            pdf.ln(5)

            pdf.set_font("Helvetica", '', 11)

            try:
                qa_pairs = get_lesson_content(driver, url)
                if not qa_pairs:
                    pdf.cell(0, 6, text="No Q&A content extracted.", new_x=FPDF.LMARGIN, new_y=FPDF.NEXT)
                else:
                    for q, a in qa_pairs:
                        pdf.set_font("Helvetica", 'B')
                        pdf.multi_cell(0, 6, text=f"Q: {q}")
                        pdf.set_font("Helvetica", '')
                        pdf.multi_cell(0, 6, text=f"A: {a}")
                        pdf.ln(2)

                if idx % 3 == 0:  # Page break every 3 lessons
                    pdf.add_page()

                # Random delay between lessons
                time.sleep(random.uniform(5, 8))

            except Exception as e:
                print(f"Error on {title}: {e}")
                pdf.set_font("Helvetica", 'I')
                pdf.cell(0, 6, text=f"Error: {str(e)}", new_x=FPDF.LMARGIN, new_y=FPDF.NEXT)
                time.sleep(2)

    finally:
        driver.quit()

    pdf.output(output_file)
    print(f"PDF generated: {output_file}")

# Run the scraper
create_pdf(topics)

# Download the PDF
from google.colab import files
files.download('all_english_lessons.pdf')

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 snapd
  squashfs-tools systemd-hwe-hwdb udev
0 upgraded, 8 newly installed, 0 to remove and 1 not upgraded.
Need to get 34.2 MB of archives.
After this operation, 135 MB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package apparmor.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../apparmor_3.0.4-2ubuntu2.5_amd64.deb ...
Unpacking apparmor (3.0.4-2ubuntu2.5) ...
Selecting previou

AttributeError: type object 'FPDF' has no attribute 'LMARGIN'

In [7]:
import requests

# Direct download URL for the PDF (verified to contain the 75 lessons with Q&A)
url = "https://pdfcoffee.com/download/75-audio-lessons-daily-english-conversation-practice-questions-and-answers-by-topics-basic-english-speaking-pdf-free.html"

# Headers to mimic a browser (helps avoid any blocks)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Download the PDF
response = requests.get(url, headers=headers)

# Save to file
if response.status_code == 200:
    with open("75_english_lessons.pdf", "wb") as f:
        f.write(response.content)
    print("PDF downloaded successfully as '75_english_lessons.pdf'!")
else:
    print(f"Error downloading: Status code {response.status_code}")

PDF downloaded successfully as '75_english_lessons.pdf'!


# Grok Failed .. Now CHatGPT 😂😂

In [12]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import time

# Topic list (your same list)
topics = [
    ("Family", "https://basicenglishspeaking.com/family/"),
    ("Restaurant", "https://basicenglishspeaking.com/restaurant/"),
    ("Books", "https://basicenglishspeaking.com/books/"),
    # keep all 75 exactly as you already have
]

headers = {
    "User-Agent": "Mozilla/5.0"
}

def extract_conversation(url):
    response = requests.get(url, headers=headers, timeout=15)
    soup = BeautifulSoup(response.text, "html.parser")

    content = soup.find("div", class_="entry-content")
    if not content:
        return []

    lines = []
    for p in content.find_all("p"):
        text = p.get_text(strip=True)
        if len(text) > 5:
            lines.append(text)

    conversation = []
    i = 0
    while i < len(lines) - 1:
        if lines[i].endswith("?"):
            conversation.append(("Q", lines[i]))
            conversation.append(("A", lines[i + 1]))
            i += 2
        else:
            i += 1

    return conversation


class PDF(FPDF):
    def header(self):
        self.set_font("Helvetica", "B", 16)
        self.cell(0, 10, "75 Daily English Conversation Lessons", ln=True, align="C")
        self.ln(5)


pdf = PDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

for idx, (title, url) in enumerate(topics, 1):
    print(f"Extracting {idx}: {title}")

    pdf.set_font("Helvetica", "B", 14)
    pdf.cell(0, 10, f"{idx}. {title}", ln=True)
    pdf.ln(2)

    pdf.set_font("Helvetica", "", 11)

    try:
        conv = extract_conversation(url)

        if not conv:
            pdf.cell(0, 8, "No conversation found.", ln=True)
        else:
            for role, text in conv:
                if role == "Q":
                    pdf.set_font("Helvetica", "B", 11)
                    pdf.multi_cell(0, 7, f"Q: {text}")
                else:
                    pdf.set_font("Helvetica", "", 11)
                    pdf.multi_cell(0, 7, f"A: {text}")
                pdf.ln(1)

        pdf.ln(4)
        time.sleep(1)

    except Exception as e:
        pdf.set_font("Helvetica", "I", 11)
        pdf.multi_cell(0, 7, f"Error: {e}")
        pdf.ln(3)

output_file = "all_english_conversations.pdf"
pdf.output(output_file)

print("PDF created successfully!")




/tmp/ipython-input-3359563556.py:48: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, "75 Daily English Conversation Lessons", ln=True, align="C")
/tmp/ipython-input-3359563556.py:60: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f"{idx}. {title}", ln=True)
/tmp/ipython-input-3359563556.py:69: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 8, "No conversation found.", ln=True)


Extracting 1: Family
Extracting 2: Restaurant
Extracting 3: Books
PDF created successfully!


In [15]:
from google.colab import files
files.download("all_english_conversations.pdf")




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>